In [1]:
import pandas as pd
import rdflib

g=rdflib.Graph()

g.load('recipe.owl', format='ttl')
g.load('neapolitan_pizza.ttl', format='ttl')

sparql_prefixes = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX neapolitan_pizza: <http://www.example.org/neapolitan_pizza#>
BASE <http://www.semanticweb.org/tpfliss/ontologies/2020/0/recipes#>
"""

In [2]:
recipes = pd.DataFrame(g.query(sparql_prefixes + """
SELECT DISTINCT ?recipe
WHERE {
    [] a :Recipe ;
    rdfs:label ?recipe .
}
""")).fillna('')

recipes

,0
0,neapolitan pizza dough


In [8]:
ingredients = pd.DataFrame(g.query(sparql_prefixes + """
SELECT DISTINCT ?amount ?unit ?ingredient ?comment
WHERE {
    [] a :Recipe ;
       :ingredient_list_item ?m .
    ?m :ingredient [ rdfs:label ?ingredient ] .
    OPTIONAL {
        ?m :amount ?amount .
    } OPTIONAL {
        ?m :unit [ rdfs:label ?unit ] .
    } OPTIONAL {
        ?m rdfs:comment ?comment .
    }
}
""")).fillna('')

ingredients

,0,1,2,3
0,1,teaspoon,yeast,
1,1.75,cup,water,
2,0.25,cup,olive oil,
3,4.5,cup,flour,
4,,,cornmeal,for dusting
5,1.75,teaspoon,salt,


Note in the future we'll want to handle modifiers (large metal spoon)
Also currently recipe step ingredients are pointing to the ingredient class rather than to specific measurement instance from the ingredients list (or even a source portion divided out from the original ingredient list).

We'll want SHACL restrictions to verify the recipe model is valid, and also probably want a tool to generate a recipe model from a DSL or csv lists.

Note the use of property paths on ingredient to override rdfs label (maybe not a great idea?)

In [9]:
steps = pd.DataFrame(g.query(sparql_prefixes + """
SELECT DISTINCT ?order ?operation ?ingredient ?product ?equipment ?description ?comment
WHERE {
    ?s a :Step ;
        :order ?order .
    OPTIONAL {
        ?s rdfs:description ?description .
    }
    OPTIONAL {
        ?s :operation [ rdfs:label ?operation ] .
    }
    OPTIONAL {
        ?s :ingredient+ [ rdfs:label ?ingredient ] .
    }
    OPTIONAL {
        ?s :product [ rdfs:label ?product ] .
    }
    OPTIONAL {
        ?s :equipment [ a [ rdfs:label ?equipment ] ] .
    }
    OPTIONAL {
        ?s rdfs:comment ?comment .
    }
}
ORDER BY ?order
"""), columns=['order', 'operation', 'ingredient', 'product', 'equipment', 'description', 'comment']).fillna('')
steps.set_index(['order'], inplace=True)

def collapse_column_as_string_list(df, grp, col):
    df[col] = df.groupby(grp)[col].apply(', '.join)

    return  steps.drop_duplicates()

# Might treat these as sub-steps in the future?
steps = collapse_column_as_string_list(steps, 'order', 'ingredient')
steps = collapse_column_as_string_list(steps, 'order', 'product')
steps = collapse_column_as_string_list(steps, 'order', 'equipment')

# Getting duplicat values, hacky fix
for col in ['ingredient', 'product', 'equipment']:
    steps[col] = steps[col].apply(lambda x: list(set(x.split(', '))) if x else [])
 
steps

,operation,ingredient,product,equipment,description,comment
order,,,,,,
1,stir,"[water, salt, yeast, flour]",[dough],"[spoon, bowl]","stir together the flour, salt, and instant yeast",
2.1,line,[],[],"[parchment paper, sheet pan]",Line sheet pan with parchment paper,
2.2,mist,[olive oil],[],"[parchment paper, sheet pan]",mist the parchment paper,
2.3,sprinkle,[cornmeal],[],[sheet pan],sprinkle flour on the counter and transfer the...,
2.4,cut,[dough],[six pieces of dough],[scraper],cut the dough into six equal parts,
2.5,round,[six pieces of dough],[six balls of dough],[hands],round each piece into a ball,
2.6,move,[six balls of dough],[],[sheet pan],transfer dough balls to sheet pan,
3.1,move,[six balls of dough],[],"[sheet pan, refrigerator]",put the pan into the refrigerator,to rest the dough
3.2,rest,[six balls of dough],[],"[sheet pan, refrigerator]",rest the dough overnight,


Note need to model "six pieces of dough" as a product of step 2.4

Need to model "move" destination better in step 2.6